In [1]:
# This code is adapted from IBM Quantum Learning, Variational quantum eigensolver
# URL: https://learning.quantum.ibm.com/tutorial/variational-quantum-eigensolver

In [2]:
import numpy as np
from scipy.optimize import minimize
import matplotlib.pyplot as plt

In [3]:
import qiskit.quantum_info as qi

n_q = 6
J = np.random.uniform(-1,1,n_q-1)
h = np.random.uniform(-1,1,n_q-1)
mu = 1

H_A = []
for i in range(n_q-1):
    j = i+1
    A = qi.SparsePauliOp.from_sparse_list(
        [("ZZ", [i, j], J[i])], num_qubits=n_q
    )
    H_A.append(A)

H_B = []
for i in range(n_q):
    B = qi.SparsePauliOp.from_sparse_list(
        [("Z", [i], mu)], num_qubits=n_q
    )
    H_B.append(B)


H = sum(H_A) + sum(H_B)

In [4]:
from qiskit.circuit.library import EfficientSU2
ansatz = EfficientSU2(n_q)

In [5]:
from qiskit_aer import AerSimulator
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager

backend = AerSimulator()
target = backend.target
pm = generate_preset_pass_manager(target=target, optimization_level=3)
ansatz_isa = pm.run(ansatz)
H_isa = H.apply_layout(layout=ansatz_isa.layout)

In [6]:
def cost_func(params, ansatz, hamiltonian, estimator):
    """Return estimate of energy from estimator

    Parameters:
        params (ndarray): Array of ansatz parameters
        ansatz (QuantumCircuit): Parameterized ansatz circuit
        hamiltonian (SparsePauliOp): Operator representation of Hamiltonian
        estimator (EstimatorV2): Estimator primitive instance
        cost_history_dict: Dictionary for storing intermediate results

    Returns:
        float: Energy estimate
    """
    pub = (ansatz, [hamiltonian], [params])
    result = estimator.run(pubs=[pub]).result()
    energy = result[0].data.evs[0]

    cost_history_dict["iters"] += 1
    cost_history_dict["prev_vector"] = params
    cost_history_dict["cost_history"].append(energy)
    #print(f"Iters. done: {cost_history_dict['iters']} [Current cost: {energy}]")
    
    return energy
    

In [7]:
from qiskit_aer.primitives import EstimatorV2

estimator = EstimatorV2()

cost_history_dict = {
    "prev_vector": None,
    "iters": 0,
    "cost_history": [],
}

x0 = np.random.uniform(0,2*np.pi, ansatz.num_parameters)

In [ ]:
res = minimize(
    cost_func,
    x0,
    args=(ansatz_isa, H_isa, estimator),
    #method="cobyla",
)
E_ground_sim = res.fun
minimizer = res.x
print("E0 =", E_ground_sim)